# Create IPyParallel Profile

Borrowed from https://gist.github.com/basnijholt/c375ea2d1df6702492b619e0873d6c7c

And https://docs.bodo.ai/latest/source/ipyparallel.html

And lotsa lotsa googling.

Overtime this should be automated with QHub HPC and provide better slurm submission scripts

In [12]:
! ipython profile create --parallel --profile=slurm

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
[ProfileCreate] Generating default config file: '/home/dharhas/.ipython/profile_slurm/ipython_config.py'
[ProfileCreate] Generating default config file: '/home/dharhas/.ipython/profile_slurm/ipython_kernel_config.py'
[ProfileCreate] Generating default config file: '/home/dharhas/.ipython/profile_slurm/ipcontroller_config.py'
[ProfileCreate] Generating default config file: '/home/dharhas/.ipython/profile_slurm/ipengine_config.py'
[ProfileCreate] Generating default config file: '/home/dharhas/.ipython/profile_slurm/ipcluster_config.py'


In [13]:
%%writefile ~/.ipython/profile_slurm/ipcontroller_config.py

c.HubFactory.ip = u'*'
c.HubFactory.registration_timeout = 600

Overwriting /home/dharhas/.ipython/profile_slurm/ipcontroller_config.py


In [14]:
%%writefile ~/.ipython/profile_slurm/ipengine_config.py

c.IPEngineApp.wait_for_url_file = 300
c.EngineFactory.timeout = 300

Overwriting /home/dharhas/.ipython/profile_slurm/ipengine_config.py


In [15]:
#HOSTFILE=$(pwd)/hostfile.$SLURM_JOB_ID
#scontrol show hostnames > $HOSTFILE

In [22]:
%%writefile ~/.ipython/profile_slurm/ipcluster_config.py

c.IPClusterStart.controller_launcher_class = 'SlurmControllerLauncher'
c.IPClusterEngines.engine_launcher_class = 'SlurmEngineSetLauncher'

# this is based on 4GB per core
c.SlurmEngineSetLauncher.batch_template = """#!/bin/sh
#SBATCH --ntasks={n}
#SBATCH --job-name=ipy-engine-
#SBATCH --mem=4G
#SBATCH --cpus-per-task=1

HOSTFILE=$(pwd)/hostfile.$SLURM_JOB_ID
scontrol show hostnames > $HOSTFILE

mpiexec -n {n} -machinefile $HOSTFILE python -m ipyparallel.engine --mpi --profile-dir ~/.ipython/profile_slurm --cluster-id ''
"""

Overwriting /home/dharhas/.ipython/profile_slurm/ipcluster_config.py
